# Preprocessing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import os
from datetime import datetime, timedelta

import warnings

warnings.filterwarnings('ignore')

Пропишем требуемые пути и откроем данные

In [2]:
cwd = os.getcwd()

PATH_SALES_DF_TRAIN = cwd + '/data/raw/sales_df_train.csv'
PATH_PR_DF = cwd + '/data/raw/pr_df.csv'
PATH_ST_DF = cwd + '/data/raw/st_df.csv'

PATH_TO_SAVE_HOLIDAYS = cwd + '/data/holidays.csv'

PATH_TO_SAVE_TRAIN_DF = cwd + '/data/preprocessing/preproc_df_train.csv'
PATH_TO_SAVE_TEST_DF = cwd + '/data/preprocessing/test.csv'
PATH_TO_SAVE_TRAIN_TEST_DF = cwd + '/data/preprocessing/train_test.csv'

In [3]:
sales_df_train = pd.read_csv(PATH_SALES_DF_TRAIN)
pr_df = pd.read_csv(PATH_PR_DF)
st_df = pd.read_csv(PATH_ST_DF)

Поскольку предсказывать будем только общее количество продаж товара(pr_sales_in_units), то в датасете sales_df_train оставим только столбцы st_id, pr_sku_id (для джойна с другими датасетами), столбец с датой date и целевой признок pr_sales_in_units

In [4]:
sales_df_train = sales_df_train[['st_id', 'pr_sales_type_id', 'pr_sku_id', 'date', 'pr_sales_in_units']]

## Предообработка данных по продажам

Создадим функцию, которая почистит данные, восстановит пропуски в датах и заполнит их нулями (решать будем через словрь)

In [5]:
def get_clear_df(df, df_date):
    new_df = df.copy(deep=True)
    # оставим в датасете только значения с целевым признаком больше или равных 0
    new_df = new_df[new_df['pr_sales_in_units']>=0].reset_index(drop=True)
    if new_df.shape[0] > 0:
        #сделаем словарь для заполнения пропусков    
        list_col = list(new_df.columns)
        list_col.remove('pr_sales_in_units')
        dict_for_fillna = {k: new_df.loc[0, k] for k in list_col}

        # присоединим датасет с датами
        new_df = new_df.merge(df_date, on='date', how='right')
        
        new_df['pr_sales_in_units'] = new_df['pr_sales_in_units'].fillna(0)
        new_df = new_df.sort_values('date').reset_index(drop=True)
                
        # заполним пропуски, отступаем последние 60 дней (большая их часть будет отброшена за счёт пропусков в скользящих средних)
        # заполняем только первые 10 значений, чтобы модель не стремилась предсказывать везде 0
        for k, v in dict_for_fillna.items():
            new_df.loc[60:,k] = new_df.loc[60:,k].fillna(v, limit=10)        
                   
    else:
        new_df = None
    
    return new_df

Напишем функцию, которая привёдт дату к нормальному формату и выделит ряд некоторые признаки

In [6]:
def get_date_and_weekday(df):
    #приведём дату в нужный формат, укажем новый индекс и день недели
    df['date'] = pd.to_datetime(df['date'])
    df['weekday'] = df['date'].dt.weekday 
    df['weekend'] = (df['weekday'] == 5) | (df['weekday'] == 6)
    # преобразуем день недели через тригонометрическую функцию    
    df['weekday_cos'] =  np.cos((2 * np.pi) / 7 * (df['weekday'] + 1))
    # аналогично поступим с неделями
    df['week'] = df['date'].dt.isocalendar().week
    df['week'] =  np.cos((2 * np.pi) / 53 * (df['week'] + 1))
    
    df.index = df['date']
    df = df.drop('date', axis=1)

    return df

Создадим функцию которая будет возвращать средню цену в определённый день за какое-то количество предшествующих недель, в дальнейшем будем получать данные за 4 недели.

In [7]:
def get_mean_in_day(df, weekday_column, column, n_week):
    list_weekday_column = df[weekday_column].unique()
    for day in list_weekday_column: 
        new_name_mean = f'mean_in_weekday_{n_week}_week'  
        df.loc[(df[weekday_column]==day), new_name_mean] = (df[df[weekday_column]==day][column]
                                                           .shift(13).shift()
                                                           .rolling(n_week)
                                                           .mean()) 
    df = df.drop(weekday_column, axis=1)
    return df

Создадим функцию для получения скользящих статистик по каждой из группировок

In [8]:
def get_rolling(df, column, n_day_list):    
    for n_day in n_day_list:
        new_name_mean = f'rolling_mean_{n_day}'
        new_name_max = f'rolling_max_{n_day}'
        new_name_min = f'rolling_min_{n_day}'        
        new_name_max_min = f'rolling_max_min_{n_day}'
        new_name_ratio = f'rolling_ratio_{n_day}'     
        df[new_name_mean] = (df[column]
                               .shift(13).shift()
                               .rolling(n_day)
                               .mean())
        df[new_name_max] = (df[column]
                               .shift(13).shift()
                               .rolling(n_day)
                               .max())
        df[new_name_min] = (df[column]
                               .shift(13).shift()
                               .rolling(n_day)
                               .min())
        df[new_name_max_min] = (df[new_name_max] + df[new_name_min]) / 2
        
        df.loc[df[new_name_mean]==0, new_name_mean] = 0.001 #защита от деления на 0
        df[new_name_ratio] = df[new_name_max_min] / df[new_name_mean]
        df
    return df

Создадим функцию для получения лагов в каждой из группировок. Будем добавлять в датасет лаги в от 1 до 14 дней, затем также посчитаем среднее по лагам в 3 дня по смещениям.

In [9]:
def get_lag(df, column, n_day_list):
    for n_day in n_day_list:
        new_name = f'lag_{n_day}'
        df[new_name] = (df[column].shift(13).shift(n_day)) 
                
    df['mean_week_lag'] = df[['lag_5', 'lag_6', 'lag_7']].mean(axis=1)
    df['ratio_lag_1_to_mean_week_lag'] = df['lag_1'] / df['mean_week_lag']
    df['ratio_lag_1_to_mean_week_lag'] = df['ratio_lag_1_to_mean_week_lag'].fillna(0)
    return df

Создадим функцию для обработки нового года, пасхи и праздников. 

In [10]:
def get_features_ny_e_h(df, list_holidays, before_2days_holidays_list):
    #Добавим флаг нового года и пасхи
    df['new_year'] = df.index=='2023-01-01'
    df['easter'] = df.index=='2023-04-16'
    #
    df['week_after_new_year'] = (df.index > '2023-01-01') & (df.index <= '2023-01-08')
    df['week_after_easter'] = (df.index > '2023-04-16') & (df.index <= '2023-01-23')
    # Добавим флаг после нового года и пасхи
    df['week_before_new_year'] = (df.index > '2022-12-24') & (df.index < '2023-01-01')
    df['week_before_easter'] = (df.index > '2023-04-09') & (df.index <= '2023-04-16')

    #Обработаем список праздников
    df['holiday'] = df.index.isin(list_holidays)
    # получим данные о днях, которые являются двумя днями, предшествующими праздникам
    df['before_2days_holidays'] = df.index.isin(before_2days_holidays_list)
    return df

Создадим список с праздничными днями с учётом переносов

In [11]:
list_holidays = [
    '2022-01-01',
    '2022-01-02',
    '2022-01-03',
    '2022-01-04',
    '2022-01-05',
    '2022-01-06',
    '2022-01-07',
    '2022-01-08',
    '2022-02-23',
    '2022-03-08',
    '2022-05-01',
    '2022-05-09',
    '2022-06-12',
    '2022-11-04',
    '2022-05-03',
    '2022-05-10',
    '2022-03-07', 
    '2023-01-01',
    '2023-01-02',
    '2023-01-03',
    '2023-01-04',
    '2023-01-05',
    '2023-01-06',
    '2023-01-07',
    '2023-01-08',
    '2023-02-23',
    '2023-03-08',
    '2023-05-01',
    '2023-05-09',
    '2023-06-12',
    '2023-11-04',
    '2023-02-24',
    '2023-05-08',
]

Сохраним данный список для дальнейшего использования в модели

In [12]:
df_holidays = pd.DataFrame({'holidays': list_holidays})
df_holidays.to_csv(PATH_TO_SAVE_HOLIDAYS, index=False)

Напишем функцию, которая будет получать даты за 2 дня до праздников, при этом продолжительные праздники будет считать одним длинным

In [13]:
def get_list_before_2days_holidays(list_holidays, n_day_before=2):
    before_2days_holidays = [list_holidays[0]]
    for i in range(1, len(list_holidays)):
        last_date = datetime.strptime(list_holidays[i - 1], '%Y-%m-%d') 
        date = datetime.strptime(list_holidays[i], '%Y-%m-%d')
        if last_date != date - timedelta(days = 1):
            new_list_day_befor = []
            for i in range(1, n_day_before + 1):
                new_date = date - timedelta(days = i)
                new_date = datetime.strftime(new_date, '%Y-%m-%d')
                new_list_day_befor.append(new_date)
            before_2days_holidays += new_list_day_befor
    return before_2days_holidays

Переименуем в таргет

In [14]:
def rename_target(df, column):
    df = df.rename(columns = {column: 'target'})    
    return df

Поскольку данных много и для данных для предсказаний функции извлечения признаков на полном датасете будут работать долго, то создадим функцию для создания словаря датасетов, что значительно увеличит скорость работы.

In [15]:
def get_dict(df, 
             df_date,
            column = 'pr_sales_in_units',
            weekday_column = 'weekday',
            n_week_for_lag = 4,
            n_day_rolling_list = [7, 14, 30],
            n_day_lag_list = list(range(1,15)),
            list_holidays = []):
    #согласно тз оставим только товары без промо
    df = df[df['pr_sales_type_id']==0]
    df = df.drop('pr_sales_type_id', axis=1)
    df['st_sku'] =  df['st_id'] + '_' + df['pr_sku_id']    
    dict_df = {}    
    before_2days_holidays_list = get_list_before_2days_holidays(list_holidays)
    for x in tqdm(df['st_sku'].unique()):
        new_df = df.loc[df['st_sku']==x].copy(deep=True).reset_index(drop=True)
        new_df = get_clear_df(new_df, df_date)
        if not new_df is None:         
            new_df = get_date_and_weekday(new_df)  
            new_df = get_mean_in_day(new_df, weekday_column, column, n_week_for_lag)
            new_df = get_rolling(new_df, column, n_day_rolling_list)
            new_df = get_lag(new_df, column, n_day_lag_list)
            new_df = get_features_ny_e_h(new_df, list_holidays, before_2days_holidays_list)
            new_df = rename_target(new_df, column)            
            new_df = new_df.drop('st_sku', axis=1)      
            
            dict_df[x] = new_df
    
    return dict_df

Напишем функцию, которая на вход будет принимать датасет, преобразовывать вызывать функцию преобразвоания в словарь и получать готовый результат.

In [16]:
def get_features_for_ts(df, 
                        column = 'pr_sales_in_units',
                        weekday_column = 'weekday',
                        n_week_for_lag = 4,
                        n_day_rolling_list = [7, 14, 30],
                        n_day_lag_list = list(range(1,15)),
                        list_holidays = []):
    print('Начата генерация признаков')
    df = df.copy(deep = True)       
    df_date = pd.DataFrame(data=df['date'].copy(deep=True).unique(), columns=['date'])
    dict_df = get_dict(df,
                       df_date,
                       column = column,
                       weekday_column = weekday_column,
                       n_week_for_lag = n_week_for_lag,
                       n_day_rolling_list = n_day_rolling_list,
                       n_day_lag_list = n_day_lag_list,
                       list_holidays = list_holidays)
       
    return pd.concat(dict_df.values(),axis=0).sort_values(['date','st_id', 'pr_sku_id'])

In [17]:
df_ts = get_features_for_ts(sales_df_train, list_holidays=list_holidays)

Начата генерация признаков


100%|██████████████████████████████████████████████████████████████████████████████| 5995/5995 [05:08<00:00, 19.40it/s]


Выведем предобработанный датасет

In [18]:
df_ts.head()

,st_id,pr_sku_id,target,weekend,weekday_cos,week,mean_in_weekday_4_week,rolling_mean_7,rolling_max_7,rolling_min_7,...,mean_week_lag,ratio_lag_1_to_mean_week_lag,new_year,easter,week_after_new_year,week_after_easter,week_before_new_year,week_before_easter,holiday,before_2days_holidays
date,,,,,,,,,,,,,,,,,,,,,
2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,1.0,False,0.62349,-0.794854,NaN,NaN,NaN,NaN,...,NaN,0.0,False,False,False,False,False,False,False,False
2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,4.0,False,0.62349,-0.794854,NaN,NaN,NaN,NaN,...,NaN,0.0,False,False,False,False,False,False,False,False
2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,033013f94a18c066e8b3d610bed34bee,4.0,False,0.62349,-0.794854,NaN,NaN,NaN,NaN,...,NaN,0.0,False,False,False,False,False,False,False,False
2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,04bbb07b1057b09d04209991f3eadd8f,0.0,False,0.62349,-0.794854,NaN,NaN,NaN,NaN,...,NaN,0.0,False,False,False,False,False,False,False,False
2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,0570ab7d07a4f2587f1ad4c4ed77e333,0.0,False,0.62349,-0.794854,NaN,NaN,NaN,NaN,...,NaN,0.0,False,False,False,False,False,False,False,False


## Предообработка данных по товарам

Согласно EDA нам необходимо сделать метку для отнесения товара к той или иной категории, согласно динамике цен на них добавим такой столбец в датасет pr_df. Создадим соответствующую функцию.

In [19]:
def get_cat(df):
    df.loc[df['pr_cat_id']=='c559da2ba967eb820766939a658022c8', 'group_cat'] = 'cat_1'
    df.loc[df['pr_subcat_id']=='60787c41b04097dfea76addfccd12243', 'group_cat'] = 'cat_2'
    df.loc[df['pr_subcat_id']=='ca34f669ae367c87f0e75dcae0f61ee5', 'group_cat'] = 'cat_3'
    df.loc[df['pr_cat_id'].isin(['e58cc5ca94270acaceed13bc82dfedf7', 
                                          'fb2fcd534b0ff3bbed73cc51df620323']), 'group_cat'] = 'cat_4'
    df.loc[df['pr_cat_id'].isin(['3de2334a314a7a72721f1f74a6cb4cee', 
                                          'f3173935ed8ac4bf073c1bcd63171f8a',
                                          'b59c67bf196a4758191e42f76670ceba']), 'group_cat'] = 'cat_5'
    df.loc[df['pr_cat_id'].isin(['28fc2782ea7ef51c1104ccf7b9bea13d', 
                                          '9701a1c165dd9420816bfec5edd6c2b1', 
                                          '5caf41d62364d5b41a893adc1a9dd5d4', 
                                          '186a157b2992e7daed3677ce8e9fe40f', 
                                          '2df45244f09369e16ea3f9117ca45157', 
                                          '6d9c547cf146054a5a720606a7694467', 
                                          '535ab76633d94208236a2e829ea6d888', 
                                          'a6ea8471c120fe8cc35a2954c9b9c595']), 'group_cat'] = 'cat_6'
    df.loc[df['pr_cat_id']=='f9ab16852d455ce9203da64f4fc7f92d', 'group_cat'] = 'cat_7'
    df.loc[df['pr_cat_id'].isin(['b7087c1f4f89e63af8d46f3b20271153', 
                                          'f93882cbd8fc7fb794c1011d63be6fb6']), 'group_cat'] = 'cat_8'
    df.loc[df['pr_cat_id']=='faafda66202d234463057972460c04f5', 'group_cat'] = 'cat_9'
    df.loc[df['pr_cat_id']=='fd5c905bcd8c3348ad1b35d7231ee2b1', 'group_cat'] = 'cat_10'
    df.loc[df['pr_cat_id']=='c9f95a0a5af052bffce5c89917335f67', 'group_cat'] = 'cat_11'
    df['group_cat'] = df['group_cat'].fillna('cat_12')
    df['pr_uom_id'] = df['pr_uom_id']==1
    df = df.drop(['pr_cat_id', 'pr_subcat_id', 'pr_group_id'], axis=1)    
    return df

In [20]:
pr_df = get_cat(pr_df)

In [21]:
pr_df.head()

,pr_sku_id,pr_uom_id,group_cat
0,fd064933250b0bfe4f926b867b0a5ec8,False,cat_3
1,71c9661741caf40a92a32d1cc8206c04,False,cat_1
2,00b72c2f01a1512cbb1d3f33319bac93,False,cat_12
3,9bc40cd2fe4f188f402bb41548c5e15c,False,cat_3
4,3a74a370c8eb032acb11ad9119242b8f,False,cat_1


## Предообработка данных по магазинам

Добавим признак среднего количества проданных товаров в магазине

In [22]:
def get_mean_sale_group(df, sales_df_train):
    df_st_mean = (sales_df_train.groupby('st_id')['pr_sales_in_units'].agg('mean')
                      .reset_index(drop=False)
                      .sort_values(by='pr_sales_in_units'))
    df = df.merge(df_st_mean, on ='st_id')
    
    df['mean_seale'] = np.nan
    df.loc[df['pr_sales_in_units'] < 2.5, 'mean_seale'] = 'mean_seale_1'
    df.loc[(df['pr_sales_in_units'] >= 2.5) & (df['pr_sales_in_units'] < 4), 'mean_seale'] = 'mean_seale_2'
    df.loc[(df['pr_sales_in_units'] >= 4) & (df['pr_sales_in_units'] < 5), 'mean_seale'] = 'mean_seale_3'
    df.loc[(df['pr_sales_in_units'] >= 5), 'mean_seale'] = 'mean_seale_4'
  
    df = df.drop('pr_sales_in_units', axis=1)
    return df

Напишем функцию, которая будет высчитывать долю товаров, продаваемых по акции

In [23]:
def get_ratio_promo(df, sales_df_train):
    df_st_ratio_promo = (sales_df_train.groupby(['st_id', 'pr_sales_type_id'])['pr_sales_in_units']
                                .agg('sum')
                                .reset_index(drop=False))
    df_st_ratio_promo = (df_st_ratio_promo.pivot(columns = 'pr_sales_type_id', index = 'st_id', values = 'pr_sales_in_units')
                            .reset_index(drop=False))

    df_st_ratio_promo['ratio_promo'] = df_st_ratio_promo[1] / df_st_ratio_promo[0]

    df = df.merge(df_st_ratio_promo[['st_id', 'ratio_promo']], on ='st_id')
    return df

Воспользуемся функциями написанной в EDA для получения скользящего среднего по всем ТЦ в датасете и рассчёта отношения летних продаж к зимним, чтобы можно было их разбить по группам

In [24]:
def get_df_ts_store(df, store_columns):
    df_st_id = df.groupby(['date',store_columns])['pr_sales_in_units'].agg('sum').reset_index(drop=False)
    df_st_id.index = df_st_id['date']
    df_st_id = df_st_id.drop('date', axis=1)
    return df_st_id

In [25]:
def get_rolling_mean(df, group_column, column):
    list_group_column = df[group_column].unique()
    for gr_col in list_group_column:
        new_name = f'rolling_mean_{column}'
        df.loc[df[group_column]==gr_col, new_name] = (df[df[group_column]==gr_col][column]
                                                                    .shift(13).shift()
                                                                    .rolling(30)
                                                                    .mean())
    df = df.drop(column, axis=1)
    return df

In [26]:
def get_ratio_summer_winter(df):
    df_july = df.loc['2023-07-01'][['st_id', 'rolling_mean_pr_sales_in_units']]
    df_jan = df.loc['2023-01-01'][['st_id', 'rolling_mean_pr_sales_in_units']]
    new_df = df_july.merge(df_jan, on='st_id', how='left')
    new_df.loc[new_df['rolling_mean_pr_sales_in_units_y']==0, 'rolling_mean_pr_sales_in_units_y'] =0.01 #защита от деления на 0
    new_df['ratio_summer_winter'] = (new_df['rolling_mean_pr_sales_in_units_x']
                                     / new_df['rolling_mean_pr_sales_in_units_y'])
    return new_df[['st_id', 'ratio_summer_winter']]

Напишем функцию проводящую полную обработку датасета

In [27]:
def get_st_df(df, sales_df_train):
    # Оставим только активные магазины и удалим столбец st_is_active
    df = df.copy(deep=True)
    df = df[df['st_is_active']!=0]
    df = df.drop('st_is_active', axis=1)  
    
    df = get_mean_sale_group(df, sales_df_train)
    df = get_ratio_promo(df, sales_df_train)
    df_st_id = get_df_ts_store(sales_df_train, 'st_id')
    df_st_id = get_rolling_mean(df_st_id, 'st_id', 'pr_sales_in_units')
    
    df_st_id_1 = df_st_id.groupby('st_id')['rolling_mean_pr_sales_in_units'].agg('max').reset_index(drop=False)
    df_st_id_2 = get_ratio_summer_winter(df_st_id)
    df_st_id = df_st_id_1.merge(df_st_id_2, on='st_id', how='left')
    
    df_st_id.loc[df_st_id['rolling_mean_pr_sales_in_units']<500,'group_shop'] = 'group_1'
    df_st_id.loc[df_st_id['ratio_summer_winter']>1,'group_shop'] = 'group_2'
    df_st_id['group_shop'] = df_st_id['group_shop'].fillna('group_3')
    df_st_id = df_st_id.drop(['rolling_mean_pr_sales_in_units', 'ratio_summer_winter'], axis=1)      
    
    df = df.merge(df_st_id, on ='st_id')
    
    return df

In [28]:
st_df = get_st_df(st_df, sales_df_train)

Выведем получившийся датасет

## Создание финального датасета

Напишем функцию для объединения датасетов магазинов и покупок.

In [29]:
def combine_shops_sales(st_df, df_ts, pr_df):
    #объединим получившиеся датасеты, перезададим индексы и удалим пропуски в отсутсвующих торговых центрах
    df_ts['date'] = df_ts.index
    df = df_ts.merge(st_df, on ='st_id', how='left')
    df = df.merge(pr_df, on ='pr_sku_id', how='left')
    df.index = df['date']
    df = df.drop('date', axis=1)
    # Создадим столбец с уникальным сочитанием группы магазина и группы категории товра 
    # удалим ненужные столбцы и пропуски в данных
    df['group_shop_cat'] = df['group_shop'] + '_' + df['group_cat']
    df = df.dropna()
    df = df.drop(['group_shop', 'group_cat'], axis=1)
    
      # преобразуем формат столбцов 
    df['st_type_format_id'] = df['st_type_format_id'].astype('int')
    df['st_type_loc_id'] = df['st_type_loc_id'].astype('int')    
    df['st_type_size_id'] = df['st_type_size_id'].astype('int')
    return df

In [30]:
df_train = combine_shops_sales(st_df, df_ts, pr_df)

In [31]:
df_train.head()

,st_id,pr_sku_id,target,weekend,weekday_cos,week,mean_in_weekday_4_week,rolling_mean_7,rolling_max_7,rolling_min_7,...,before_2days_holidays,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,mean_seale,ratio_promo,pr_uom_id,group_shop_cat
date,,,,,,,,,,,,,,,,,,,,,
2022-11-28,16a5cdae362b8d27a1d8f8c7b78b4330,01e4734745e97e52d3213449e1a05dd7,3.0,False,0.62349,0.889657,3.0,3.571429,7.0,1.0,...,False,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,mean_seale_4,0.704237,False,group_2_cat_5
2022-11-28,16a5cdae362b8d27a1d8f8c7b78b4330,02713435f3587e2c81d8f6a9016763ea,1.0,False,0.62349,0.889657,0.0,0.001000,0.0,0.0,...,False,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,mean_seale_4,0.704237,True,group_2_cat_12
2022-11-28,16a5cdae362b8d27a1d8f8c7b78b4330,040a02c2ad1561cbcfc9cae5b4c4b73b,1.0,False,0.62349,0.889657,0.0,0.285714,1.0,0.0,...,False,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,mean_seale_4,0.704237,True,group_2_cat_6
2022-11-28,16a5cdae362b8d27a1d8f8c7b78b4330,0448c8afc036bb44c457d7e9edd74c50,1.0,False,0.62349,0.889657,0.0,0.285714,1.0,0.0,...,False,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,mean_seale_4,0.704237,True,group_2_cat_12
2022-11-28,16a5cdae362b8d27a1d8f8c7b78b4330,04bbb07b1057b09d04209991f3eadd8f,0.0,False,0.62349,0.889657,0.0,0.142857,1.0,0.0,...,False,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,mean_seale_4,0.704237,True,group_2_cat_12


In [32]:
df_train.to_csv(PATH_TO_SAVE_TRAIN_DF)

## Создание объединённой функции препроцессинга

Напишем функцию, которая будет обрезать датасет по тестовому диапазону, либо генерировать заготовку для прогноза по последней дате датасета.

In [33]:
def get_test_df(df, first_date = None):
    # Если дата среза неизвестна, тогда берём последнюю дату в датасете
    if first_date is None:
        first_date = df['date'].max()
        
    unique_st_pr = df[df['pr_sales_type_id']==0].copy(deep=True)
    unique_st_pr = unique_st_pr[['st_id', 'pr_sku_id', 'pr_sales_type_id']].drop_duplicates()
    
    # получим заготовку для предсказаний (будем её делать с завтрашнего дня, поэтому диапазон от 1 до 15)
    df_list = [] 
    for i in range(1, 15):
        date = datetime.strptime(first_date, '%Y-%m-%d') + timedelta(days = i)
        date = datetime.strftime(date, '%Y-%m-%d')
        new_df = unique_st_pr.copy(deep=True)
        new_df['pr_sales_in_units'] = 0
        new_df['date'] = date
        df_list.append(new_df)
    new_df = pd.concat(df_list, axis=0)
    
    #добавим информацию по предыдущему периоду     
    old_df = df[['st_id', 'pr_sales_type_id', 'pr_sku_id', 'date', 'pr_sales_in_units']]
    
    return pd.concat([old_df, new_df], axis=0).reset_index(drop=True)

Создадим объединённую функцию, которая на вход будет принимать сырые данные по продажам, обработанные данные по категориям товаров для ML модели, обработанные данные по торговым центрам и возвращать обработанный датасет

In [34]:
def preproceccing_df(df, 
                     pr_df, 
                     st_df, 
                     first_date = None,
                     is_train = True, 
                     column = 'pr_sales_in_units',
                     weekday_column = 'weekday',
                     n_week_for_lag = 4,
                     n_day_rolling_list = [7, 14, 30],
                     n_day_lag_list = list(range(1,15)),
                     list_holidays = []):
    pr_df = get_cat(pr_df)
    st_df = get_st_df(st_df, df)  
    
    if first_date is not None:
        #если указана первая дата перед прогнозом, то обрезаем датасет
        df = df.copy(deep=True)
        df.index = pd.to_datetime(df['date'])        
        df = df.sort_index()
        df = df.loc[:first_date]
            
    if first_date is None:
        # eсли дата среза неизвестна, тогда берём последнюю дату в датасете
        first_date = df['date'].max()  
        
    if not is_train:
        #если не тренировочный, то получаем данные через функцию
        df = get_test_df(df, first_date = first_date)
    
    df = df[['st_id', 'pr_sales_type_id', 'pr_sku_id', 'date', 'pr_sales_in_units']]
    df = get_features_for_ts(df, 
                             column = column,
                             weekday_column = weekday_column,
                             n_week_for_lag = n_week_for_lag,
                             n_day_rolling_list = n_day_rolling_list,
                             n_day_lag_list = n_day_lag_list,
                             list_holidays = list_holidays)
    if not is_train:
        first_date = datetime.strptime(first_date, '%Y-%m-%d') + timedelta(days = 1)
        first_date = datetime.strftime(first_date, '%Y-%m-%d')
        df = df.loc[first_date:]
    df = combine_shops_sales(st_df, df, pr_df)
    return df

Протестируем её работу на данных для теста

In [35]:
sales_df_train = pd.read_csv(PATH_SALES_DF_TRAIN)
pr_df = pd.read_csv(PATH_PR_DF)
st_df = pd.read_csv(PATH_ST_DF)

Для тестового набора найдм дату среза (сделаем 14 дней для прогноза + 1 день, чтобе получить предшествующую дату)

In [36]:
date = sales_df_train['date'].max()
first_date = datetime.strptime(date, '%Y-%m-%d') - timedelta(days = 15)
first_date = datetime.strftime(first_date, '%Y-%m-%d')

In [37]:
%%time
df_test = preproceccing_df(sales_df_train, 
                           pr_df, 
                           st_df,
                           first_date,
                           list_holidays = list_holidays, 
                           is_train=False)

Начата генерация признаков


100%|██████████████████████████████████████████████████████████████████████████████| 5881/5881 [05:30<00:00, 17.79it/s]


CPU times: total: 5min 35s
Wall time: 5min 34s


In [38]:
%%time
df_train_test = preproceccing_df(sales_df_train, 
                               pr_df, 
                               st_df,
                               first_date,
                               list_holidays = list_holidays, 
                               is_train=True)

Начата генерация признаков


100%|██████████████████████████████████████████████████████████████████████████████| 5881/5881 [05:02<00:00, 19.44it/s]


CPU times: total: 5min 3s
Wall time: 5min 8s


In [39]:
df_test.head()

,st_id,pr_sku_id,target,weekend,weekday_cos,week,mean_in_weekday_4_week,rolling_mean_7,rolling_max_7,rolling_min_7,...,before_2days_holidays,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,mean_seale,ratio_promo,pr_uom_id,group_shop_cat
date,,,,,,,,,,,,,,,,,,,,,
2023-07-04,084a8a9aa8cced9175bd07bc44998e75,0376a60d9a7ce7965beddc4815588697,0.0,False,-0.222521,-0.984231,0.0,0.142857,1.0,0.0,...,False,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,4,3,19,mean_seale_1,1.333333,False,group_3_cat_12
2023-07-04,084a8a9aa8cced9175bd07bc44998e75,88feeeb024d3f69da7322d76b7b53744,0.0,False,-0.222521,-0.984231,0.0,0.001000,0.0,0.0,...,False,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,4,3,19,mean_seale_1,1.333333,False,group_3_cat_12
2023-07-04,084a8a9aa8cced9175bd07bc44998e75,be8d2843456cac871fc116ab25d02994,0.0,False,-0.222521,-0.984231,0.0,0.001000,0.0,0.0,...,False,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,4,3,19,mean_seale_1,1.333333,False,group_3_cat_12
2023-07-04,084a8a9aa8cced9175bd07bc44998e75,c2718cfd2edcbadfe0162a4f4c91f3a0,0.0,False,-0.222521,-0.984231,0.0,0.001000,0.0,0.0,...,False,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,4,3,19,mean_seale_1,1.333333,False,group_3_cat_12
2023-07-04,084a8a9aa8cced9175bd07bc44998e75,c4a665596d4f67cecb7542c9fad407ee,0.0,False,-0.222521,-0.984231,0.0,0.001000,0.0,0.0,...,False,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,4,3,19,mean_seale_1,1.333333,True,group_3_cat_4


In [40]:
df_train_test.tail()

,st_id,pr_sku_id,target,weekend,weekday_cos,week,mean_in_weekday_4_week,rolling_mean_7,rolling_max_7,rolling_min_7,...,before_2days_holidays,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,mean_seale,ratio_promo,pr_uom_id,group_shop_cat
date,,,,,,,,,,,,,,,,,,,,,
2023-07-03,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fa99a6fd2d3d7419c20fdd03704c5822,0.0,False,0.62349,-0.984231,0.00,0.001000,0.0,0.0,...,False,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,mean_seale_3,0.762251,True,group_2_cat_5
2023-07-03,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fafbee588d44b4a8d561d81680174dc0,0.0,False,0.62349,-0.984231,0.00,0.285714,1.0,0.0,...,False,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,mean_seale_3,0.762251,True,group_2_cat_6
2023-07-03,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fbf6c91454d7c3cea7b03f3092cbfb73,2.0,False,0.62349,-0.984231,0.75,1.142857,2.0,0.0,...,False,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,mean_seale_3,0.762251,True,group_2_cat_6
2023-07-03,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fe50ae64d08d4f8245aaabc55d1baf79,0.0,False,0.62349,-0.984231,2.25,6.571429,8.0,5.0,...,False,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,mean_seale_3,0.762251,True,group_2_cat_6
2023-07-03,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fe5d18ae6650335830e4c1dbd9e6ddb9,2.0,False,0.62349,-0.984231,1.25,1.428571,3.0,0.0,...,False,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,mean_seale_3,0.762251,False,group_2_cat_5


Сохраним получившиеся датасеты

In [41]:
df_test.to_csv(PATH_TO_SAVE_TEST_DF)

df_train_test.to_csv(PATH_TO_SAVE_TRAIN_TEST_DF)

## Выводы

В данном разделе была проведена прдобработка данных в ходе которой:
1. Произведено преобразование кодов товаров в категории согласно EDA для дальнейшего обуечние моделей
2. Созданы признаки для временного ряда (лаги, скользящие среднии, выделены дни недели, созданы флаги для выходных и праздничных дней, и ряд других признаков)
3. Почищены данные от выбрасов.
4. Проведено преобразование по информации о магазинах, удалены неактивные.
5. Создан отдельный столбец для выбора конкретной модели для конкретного временного ряда.
6. Созданы функции, позволяющие автоматизировать процес предообработки данных.